In [7]:
import json
from pathlib import Path
import tqdm as tqdm
import pandas as pd

In [20]:
pd.set_option('display.max_rows', None)       # Show all rows
pd.set_option('display.max_columns', None)    # Show all columns
pd.set_option('display.width', None)          # Don't limit the width
pd.set_option('display.max_colwidth', None)   # Show full content in each cell (for older pandas versions)
# For pandas >= 1.0.0
pd.set_option('display.max_colwidth', None)

In [42]:
log_file_path = Path("/leonardo_scratch/fast/EUHPC_D21_101/max_lue/experiments/8B_tests/checkpoints/2025-06-24__09-36-26_cb8f4503/debug/tensor_stats_rank_0.jsonl") # 128 ranks, TP = 4
log_file_path = Path("/leonardo_scratch/fast/EUHPC_D21_101/max_lue/experiments/8B_tests/checkpoints/2025-06-24__10-03-46_a95b5a74/debug/tensor_stats_rank_0.jsonl") # 32 ranks , TP = 1

In [43]:
tensor_stats_list = []
with open(log_file_path, 'r') as file:
    for line in tqdm.tqdm(file):
        try:
            data = json.loads(line)
            tensor_stats_list.append(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
        except KeyError as e:
            print(f"Key error: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

41702it [00:01, 22826.35it/s]


In [44]:
data_frame = pd.DataFrame(tensor_stats_list)
data_frame = data_frame[["counter", "hook_type", "tensor_tag", "global_shape", "local_shape",  "mean", "std", "min", "max", "nan_count", "inf_count", "rank"]]
data_frame.head()

,counter,hook_type,tensor_tag,global_shape,local_shape,mean,std,min,max,nan_count,inf_count,rank
0,0,pre_forward,transformer.wte.weight,"[50304, 4096]","[50304, 4096]",0.000002,0.02002,-0.122070,0.112793,0,0,0
1,0,forward_output,transformer.wte,"[1, 8192, 4096]","[1, 8192, 4096]",-0.000010,0.02002,-0.102051,0.112793,0,0,0
2,0,forward_output,transformer.drop,"[1, 8192, 4096]","[1, 8192, 4096]",-0.000010,0.02002,-0.102051,0.112793,0,0,0
3,0,pre_forward,transformer.h.0.attention_norm.weight,[4096],[4096],1.000000,0.00000,1.000000,1.000000,0,0,0
4,0,pre_forward,transformer.h.0.attention_norm.bias,[4096],[4096],0.000000,0.00000,0.000000,0.000000,0,0,0


In [45]:

# in order
data_frame[(data_frame["counter"] < 2) & (data_frame["hook_type"] == "pre_forward")]

,counter,hook_type,tensor_tag,global_shape,local_shape,mean,std,min,max,nan_count,inf_count,rank
0,0,pre_forward,transformer.wte.weight,"[50304, 4096]","[50304, 4096]",1.728535e-06,2.001953e-02,-1.220703e-01,1.127930e-01,0,0,0
3,0,pre_forward,transformer.h.0.attention_norm.weight,[4096],[4096],1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0,0,0
4,0,pre_forward,transformer.h.0.attention_norm.bias,[4096],[4096],0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0
6,0,pre_forward,transformer.h.0.attn.q_attn.weight,"[4096, 4096]","[4096, 4096]",-5.930662e-06,2.001953e-02,-1.040039e-01,1.166992e-01,0,0,0
8,0,pre_forward,transformer.h.0.attn.k_attn.weight,"[1024, 4096]","[1024, 4096]",6.444752e-07,2.001953e-02,-1.088867e-01,1.015625e-01,0,0,0
10,0,pre_forward,transformer.h.0.attn.v_attn.weight,"[1024, 4096]","[1024, 4096]",3.740191e-06,2.001953e-02,-9.765625e-02,9.814453e-02,0,0,0
15,0,pre_forward,transformer.h.0.attn.c_proj.weight,"[4096, 4096]","[4096, 4096]",1.564622e-07,2.502441e-03,-1.330566e-02,1.257324e-02,0,0,0
19,0,pre_forward,transformer.h.0.ffn_norm.weight,[4096],[4096],1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0,0,0
20,0,pre_forward,transformer.h.0.ffn_norm.bias,[4096],[4096],0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0
22,0,pre_forward,transformer.h.0.mlp.W.weight,"[11264, 4096]","[11264, 4096]",6.053597e-08,2.001953e-02,-1.132812e-01,1.088867e-01,0,0,0


In [46]:
# sort by max 

data_frame[(data_frame["counter"] < 2) & (data_frame["hook_type"] == "pre_forward")].sort_values(by=["min"], ascending=True).head(20)

,counter,hook_type,tensor_tag,global_shape,local_shape,mean,std,min,max,nan_count,inf_count,rank
50,0,pre_forward,transformer.h.1.mlp.W.weight,"[11264, 4096]","[11264, 4096]",2.160668e-06,0.02002,-0.122070,0.110352,0,0,0
0,0,pre_forward,transformer.wte.weight,"[50304, 4096]","[50304, 4096]",1.728535e-06,0.02002,-0.122070,0.112793,0,0,0
1502,1,pre_forward,transformer.h.1.mlp.W.weight,"[11264, 4096]","[11264, 4096]",2.160668e-06,0.02002,-0.122070,0.110352,0,0,0
1452,1,pre_forward,transformer.wte.weight,"[50304, 4096]","[50304, 4096]",1.728535e-06,0.02002,-0.122070,0.112793,0,0,0
1981,1,pre_forward,transformer.h.18.mlp.V.weight,"[11264, 4096]","[11264, 4096]",2.637506e-06,0.02002,-0.121582,0.112305,0,0,0
529,0,pre_forward,transformer.h.18.mlp.V.weight,"[11264, 4096]","[11264, 4096]",2.637506e-06,0.02002,-0.121582,0.112305,0,0,0
781,0,pre_forward,transformer.h.27.mlp.V.weight,"[11264, 4096]","[11264, 4096]",1.564622e-07,0.02002,-0.121094,0.111816,0,0,0
2233,1,pre_forward,transformer.h.27.mlp.V.weight,"[11264, 4096]","[11264, 4096]",1.564622e-07,0.02002,-0.121094,0.111816,0,0,0
1642,1,pre_forward,transformer.h.6.mlp.W.weight,"[11264, 4096]","[11264, 4096]",2.890825e-06,0.02002,-0.120117,0.109375,0,0,0
190,0,pre_forward,transformer.h.6.mlp.W.weight,"[11264, 4096]","[11264, 4096]",2.890825e-06,0.02002,-0.120117,0.109375,0,0,0


In [47]:
data_frame[(data_frame["counter"] == 1) & (data_frame["hook_type"] == "pre_forward")]

,counter,hook_type,tensor_tag,global_shape,local_shape,mean,std,min,max,nan_count,inf_count,rank
1452,1,pre_forward,transformer.wte.weight,"[50304, 4096]","[50304, 4096]",1.728535e-06,2.001953e-02,-1.220703e-01,1.127930e-01,0,0,0
1455,1,pre_forward,transformer.h.0.attention_norm.weight,[4096],[4096],1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0,0,0
1456,1,pre_forward,transformer.h.0.attention_norm.bias,[4096],[4096],6.309619e-12,3.001333e-10,-3.001333e-10,3.001333e-10,0,0,0
1458,1,pre_forward,transformer.h.0.attn.q_attn.weight,"[4096, 4096]","[4096, 4096]",-5.930662e-06,2.001953e-02,-1.040039e-01,1.166992e-01,0,0,0
1460,1,pre_forward,transformer.h.0.attn.k_attn.weight,"[1024, 4096]","[1024, 4096]",6.444752e-07,2.001953e-02,-1.088867e-01,1.015625e-01,0,0,0
1462,1,pre_forward,transformer.h.0.attn.v_attn.weight,"[1024, 4096]","[1024, 4096]",3.740191e-06,2.001953e-02,-9.765625e-02,9.814453e-02,0,0,0
1467,1,pre_forward,transformer.h.0.attn.c_proj.weight,"[4096, 4096]","[4096, 4096]",1.564622e-07,2.502441e-03,-1.330566e-02,1.257324e-02,0,0,0
1471,1,pre_forward,transformer.h.0.ffn_norm.weight,[4096],[4096],1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0,0,0
1472,1,pre_forward,transformer.h.0.ffn_norm.bias,[4096],[4096],2.046363e-12,3.001333e-10,-3.001333e-10,3.001333e-10,0,0,0
1474,1,pre_forward,transformer.h.0.mlp.W.weight,"[11264, 4096]","[11264, 4096]",6.053597e-08,2.001953e-02,-1.132812e-01,1.088867e-01,0,0,0


In [48]:
data_frame[(data_frame["counter"] == 0) & (data_frame["hook_type"] == "forward_output")].sort_values(by=["max"], ascending=False).head(20)

,counter,hook_type,tensor_tag,global_shape,local_shape,mean,std,min,max,nan_count,inf_count,rank
334,0,forward_output,transformer.h.11.mlp.V,"[1, 8192, 11264]","[1, 8192, 11264]",-0.002747,1.281250,-6.93750,8.50000,0,0,0
898,0,forward_output,transformer.h.31,"[1, 8192, 4096]","[1, 8192, 4096]",0.001503,1.546875,-8.50000,8.43750,0,0,0
870,0,forward_output,transformer.h.30,"[1, 8192, 4096]","[1, 8192, 4096]",0.001060,1.523438,-8.06250,8.25000,0,0,0
903,0,forward_output,transformer.lm_head,"[1, 8192, 50304]","[1, 8192, 50304]",-0.000412,1.281250,-7.75000,7.96875,0,0,0
595,0,forward_output,transformer.h.21.attn.q_attn,"[1, 8192, 4096]","[1, 8192, 4096]",0.000900,1.281250,-7.21875,7.90625,0,0,0
600,0,forward_output,transformer.h.21.attn.qkv_transforms.0,"[1, 32, 8192, 128]","[1, 32, 8192, 128]",0.000900,1.281250,-7.21875,7.90625,0,0,0
180,0,forward_output,transformer.h.6.attn.qkv_transforms.0,"[1, 32, 8192, 128]","[1, 32, 8192, 128]",-0.002991,1.281250,-6.81250,7.78125,0,0,0
175,0,forward_output,transformer.h.6.attn.q_attn,"[1, 8192, 4096]","[1, 8192, 4096]",-0.002991,1.281250,-6.81250,7.78125,0,0,0
842,0,forward_output,transformer.h.29,"[1, 8192, 4096]","[1, 8192, 4096]",0.000629,1.500000,-8.06250,7.75000,0,0,0
726,0,forward_output,transformer.h.25.mlp.V,"[1, 8192, 11264]","[1, 8192, 11264]",0.002304,1.281250,-7.28125,7.71875,0,0,0


In [58]:
data_frame[(data_frame["counter"] == 25) & (data_frame["hook_type"] == "backward_output")].sort_values(by=["max"], ascending=False).head(20)

,counter,hook_type,tensor_tag,global_shape,local_shape,mean,std,min,max,nan_count,inf_count,rank
37750,25,backward_output,transformer.drop,"[1, 8192, 4096]","[1, 8192, 4096]",5.343281e-11,0.000237,-0.019043,0.017456,0,0,0
37751,25,backward_output,transformer.wte,"[1, 8192, 4096]","[1, 8192, 4096]",5.343281e-11,0.000237,-0.019043,0.017456,0,0,0
37747,25,backward_output,transformer.h.0.attn,"[1, 8192, 4096]","[1, 8192, 4096]",-3.365130e-11,0.000067,-0.001373,0.001350,0,0,0
37740,25,backward_output,transformer.h.0.attn.c_proj,"[1, 8192, 4096]","[1, 8192, 4096]",-3.365130e-11,0.000067,-0.001373,0.001350,0,0,0
37739,25,backward_output,transformer.h.0.attn.resid_dropout,"[1, 8192, 4096]","[1, 8192, 4096]",-3.365130e-11,0.000067,-0.001373,0.001350,0,0,0
37737,25,backward_output,transformer.h.0.mlp,"[1, 8192, 4096]","[1, 8192, 4096]",-9.492851e-12,0.000008,-0.000633,0.000648,0,0,0
37749,25,backward_output,transformer.h.0,"[1, 8192, 4096]","[1, 8192, 4096]",-9.492851e-12,0.000008,-0.000633,0.000648,0,0,0
37733,25,backward_output,transformer.h.0.mlp.W_2,"[1, 8192, 4096]","[1, 8192, 4096]",-9.492851e-12,0.000008,-0.000633,0.000648,0,0,0
37743,25,backward_output,transformer.h.0.attn.qkv_transforms.0,"[1, 8, 8192, 128]","[1, 8, 8192, 128]",6.821210e-11,0.000003,-0.000526,0.000546,0,0,0
37744,25,backward_output,transformer.h.0.attn.v_attn,"[1, 8192, 1024]","[1, 8192, 1024]",6.821210e-11,0.000003,-0.000526,0.000546,0,0,0


In [50]:
data_frame[(data_frame["counter"] == 0) & (data_frame["hook_type"] == "backward_output")].sort_values(by=["max"], ascending=True)

,counter,hook_type,tensor_tag,global_shape,local_shape,mean,std,min,max,nan_count,inf_count,rank
931,0,backward_output,transformer.h.30.attn.qkv_transforms.0,"[1, 32, 8192, 128]","[1, 32, 8192, 128]",-1.456613e-12,4.516914e-08,-0.000001,9.685755e-07,0,0,0
936,0,backward_output,transformer.h.30.attn.q_attn,"[1, 8192, 4096]","[1, 8192, 4096]",-1.456613e-12,4.516914e-08,-0.000001,9.685755e-07,0,0,0
914,0,backward_output,transformer.h.31.attn.qkv_transforms.0,"[1, 32, 8192, 128]","[1, 32, 8192, 128]",1.068656e-11,4.423782e-08,-0.000001,1.043081e-06,0,0,0
919,0,backward_output,transformer.h.31.attn.q_attn,"[1, 8192, 4096]","[1, 8192, 4096]",1.068656e-11,4.423782e-08,-0.000001,1.043081e-06,0,0,0
948,0,backward_output,transformer.h.29.attn.qkv_transforms.0,"[1, 32, 8192, 128]","[1, 32, 8192, 128]",1.165290e-11,4.679896e-08,-0.000001,1.117587e-06,0,0,0
953,0,backward_output,transformer.h.29.attn.q_attn,"[1, 8192, 4096]","[1, 8192, 4096]",1.165290e-11,4.679896e-08,-0.000001,1.117587e-06,0,0,0
987,0,backward_output,transformer.h.27.attn.q_attn,"[1, 8192, 4096]","[1, 8192, 4096]",2.458478e-12,5.145557e-08,-0.000001,1.221895e-06,0,0,0
982,0,backward_output,transformer.h.27.attn.qkv_transforms.0,"[1, 32, 8192, 128]","[1, 32, 8192, 128]",2.458478e-12,5.145557e-08,-0.000001,1.221895e-06,0,0,0
1016,0,backward_output,transformer.h.25.attn.qkv_transforms.0,"[1, 32, 8192, 128]","[1, 32, 8192, 128]",-1.051603e-11,5.564652e-08,-0.000001,1.274049e-06,0,0,0
1021,0,backward_output,transformer.h.25.attn.q_attn,"[1, 8192, 4096]","[1, 8192, 4096]",-1.051603e-11,5.564652e-08,-0.000001,1.274049e-06,0,0,0
